# Train LSTM Model by MLFlow and PyTorch Lightning

In [ ]:
from argparse import ArgumentParser
import os

from loguru import logger
import mlflow
from mlflow.tracking import MlflowClient
import numpy as np
import pandas as pd
import pytorch_lightning as pl
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix
import torch

from models import HarLSTM, ModelUtils
from pl_data import HarDataModule
from utils import FeatUtils
from chart import plot_feat_tensor_chart, plot_conf_matrix_chart

%load_ext autoreload
%autoreload 2

# 1. Prepare features

In [ ]:
data_dir_path = "./data/har_dataset"
batch_size = 16
data_module = HarDataModule(data_dir_path, 
                            batch_size=batch_size,
                           normalize="std")

# 2. Define Network parameters

In [ ]:
lstr_args = ['--max_epochs','50',
            '--gpus', '1',
             '--batch_size', '16',
             '--stochastic_weight_avg', 'True',
             '--gradient_clip_val', '5',
             '--gradient_clip_algorithm', 'norm',
            # DEBUGGING https://pytorch-lightning.readthedocs.io/en/latest/common/debugging.html
            # don't forget to turn it off after debugging, slows things down a lot.
            # '--profiler', 'pytorch', # issue no.3
            # '--log_gpu_memory', 'all',
            # '--limit_train_batches', '3',
            # '--limit_predict_batches', '3',
            # '--overfit_batches', '3',
            # Inspect gradient norms
            # about 10% performance hit, let's do it always anyway.
            # '--track_grad_norm', '2',
             ]

parser = ArgumentParser()
parser.add_argument('--batch_size', default=16, type=int)
parser = pl.Trainer.add_argparse_args(parser)
args = parser.parse_args(lstr_args)

In [ ]:
# check if GPU is available
use_gpu = torch.cuda.is_available()
if(use_gpu):
    print('Training on GPU!')
else: 
    print('No GPU available, training on CPU; consider making n_epochs very small.')

In [ ]:
# Instantiate the model w/ hyperparams
input_size = 9
output_size = 6
n_hidden = 128
n_layers = 2

# training params
epochs = 50
lr=0.0001

In [ ]:
net = HarLSTM(input_size, output_size, n_hidden=n_hidden, n_layers=n_layers)
print("Model information:")
print(net)
trainer = pl.Trainer.from_argparse_args(args)

# 3. Train the model by MLFlow

In [ ]:
# Define helper functions
def log_model_params_step(net):
    mlflow.log_param("model_type", type(net))
    mlflow.log_param("n_layers", net.n_layers)
    mlflow.log_param("n_hidden", net.n_hidden)
    mlflow.log_param("drop_prob", net.drop_prob)
    mlflow.log_param("input_size", net.input_size)

def save_scaler_step(scaler, scaler_path="scaler.pkl"):
    FeatUtils.save_feat_scaler(scaler, scaler_path)
    mlflow.log_artifact(scaler_path, artifact_path="model")
    os.remove(scaler_path)

def show_data_sample_step(data_module, n_sample=4, fig_dir_path="figures/data_sample"):
    """Save some data as figures from each set (train/valid/test)"""
    loader_dict = {
        "train": data_module.train_dataloader(),
        "test": data_module.test_dataloader(),
        "valid": data_module.val_dataloader(),
    }
    
    for data_type, loader in loader_dict.items():
        inputs, labels = iter(loader).next()

        for i in range(n_sample):
            chart = plot_feat_tensor_chart(inputs[i], labels[i])
            chart_path = f"train_sample_{i}.html"
            chart.save(chart_path, embed_options={"renderer":"svg"})
            mlflow.log_artifact(chart_path, artifact_path=fig_dir_path)
            os.remove(chart_path)

def evaluate_model_step(net, data_module, batch_size, fig_dir_path="figures/evaluation", use_gpu=True):
    """Evaluate a model and save a confusion matrix chart"""
    test_loader = data_module.test_dataloader()
    test_loss, test_labels, preds  = ModelUtils.test_net(net, net.criterion, test_loader, batch_size, use_gpu=use_gpu)
    
    acc = accuracy_score(test_labels, preds)
    prec, recall, f1, _ = precision_recall_fscore_support(test_labels, preds, average="macro")
    
    mlflow.log_metric("acc", acc)
    mlflow.log_metric("prec", prec)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("f1", f1)
    
    # Let's save both html and png formats
    chart = plot_conf_matrix_chart(test_labels, preds)
    html_chart_path = "conf_matrix.html"
    chart.save(html_chart_path, embed_options={"renderer":"svg"})
    mlflow.log_artifact(html_chart_path, artifact_path=fig_dir_path)
    
    png_chart_path = "conf_matrix.png"
    chart.save(png_chart_path)
    mlflow.log_artifact(png_chart_path, artifact_path=fig_dir_path)
    
    os.remove(html_chart_path)
    os.remove(png_chart_path)

In [ ]:
experiment_name = "HAR_LSTM_Experiment"
mlflow_uri = "http://mlflow_tracker:5000"
mlflow.set_tracking_uri(mlflow_uri)

mlflow.set_experiment(experiment_name)

tracking_uri = mlflow.get_tracking_uri()
print("Current tracking uri: {}".format(tracking_uri))

In [ ]:
mlflow_run_name = "HAR_LSTM_Training"
mlflow.pytorch.autolog()

# Train the model
with mlflow.start_run(run_name=mlflow_run_name) as run:
    artifact_uri = mlflow.get_artifact_uri()
    print("Current artifact uri: {}".format(artifact_uri))
    
    log_model_params_step(net)
    
    mlflow.log_param("batch_size", batch_size)
    mlflow.log_param("train_val_ratio", data_module.train_val_ratio)
    mlflow.log_param("scaler", type(data_module.scaler) if data_module.scaler is not None else None)
    trainer.fit(net, datamodule=data_module)
    # Run this for calculating "test_loss" metric.
    # It will be automatically pushed to MLFlow tracker.
    trainer.test(ckpt_path="best", datamodule=data_module)
    
    show_data_sample_step(data_module)
    save_scaler_step(data_module.scaler)
    evaluate_model_step(net, data_module, batch_size, use_gpu=use_gpu)
    
    logger.info("MLFlow finished!")

# 4. Load the previously model and test manually

In [ ]:
run_id = run.info.run_id
model_uri = f"runs:/{run_id}/model"
loaded_net = mlflow.pytorch.load_model(model_uri=model_uri)

In [ ]:
test_loader = data_module.test_dataloader()
test_loss, test_labels, preds  = ModelUtils.test_net(loaded_net, loaded_net.criterion, test_loader, batch_size, use_gpu=use_gpu)

In [ ]:
acc = accuracy_score(test_labels, preds)
prec, recall, f1, _ = precision_recall_fscore_support(test_labels, preds, average="macro")

In [ ]:
print(f"accuracy: {acc}")
print(f"precision: {prec}")
print(f"recall: {recall}")
print(f"f1: {f1}")